Elements for a CO WD type Ia supernova

H1 --> 1, 1, 1.00727 (accreted from companion) \
H2 --> 2, 1, 2.0140  (accreted from companion) \
He3 --> 3, 2, 3.01603 (accreted from companion) \
He4 --> 4, 2, 4.00260 (accreted from companion) \
C12 --> 12, 6, 12. (CNO ) \
N14 -->  14, 7, 14.007 (CNO) \
O16 --> 16, 8, 15.994 (CNO) \
Ne20 --> 20, 10, 20.1797 (alpha network) \
Mg24 --> 24, 12, 24.3050 (alpha network) \
Si28 --> 28, 14, 27.976 (alpha network) \
S32 --> 32, 16, 31.972 (alpha network) \
Ar36 --> 36, 18, 35.967 (alpha network) \
Ca40 --> 40, 20, 39.962 (alpha network) \
Ti44 --> 44, 22, 43.9596 (alpha network) \
Cr48 --> 48, 24, 47.9540 (alpha network) \
Fe52 --> 52, 26, 51.9481 (alpha network) \
Fe54 --> 54, 26, 53.939 (electron capture) \
Ni56 --> 56, 28, 55.940 (alpha network) \
Co56 --> 56, 27, 55.9398393 (decay of 56Ni) \
Fe56 --> 56, 26, 55.9349 (decay of 56Co) \
Fe57 --> 57, 26, 56.9353 (decay of 58Fe) \
Fe58 --> 58, 26, 57.9332 (neutron capture) \
Ni58 --> 58, 28, 57.9353429 (neutron capture) 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
class elements:
    
    def __init__(self, A, Z, weight, repr):
        
        '''
        A: maass number
        Z: atomin number
        weight: atomic weight (in amu)
        
        '''
        
        self.A = A
        self.Z = Z
        self.weight = weight
        self.repr = repr
        
    def __repr__(self):
        
        '''
        representation object
        
        '''
        
        return '%s: A=%d, Z=%d, weight=%.3f' %(self.repr, self.A,
                                              self.Z, self.weight)
    

class Layers:
    
    def __init__(self, abundances):
        
        '''
        abundances: Abundances of the layers
        
        '''
        
        self.normalization = sum(abundances.values())
        self.abundances = {key:abundances[key]/self.normalization 
                          for key in abundances}
        
    def __repr__(self):
        
        s = 'Layer:\n'
        return s + '\n'.join(['\t%s abundance = %.3e' % (key, self.abundances[key])
                             for key in self.abundances])

In [23]:
# A supernova atmosphere

# Unburned 

C12 = elements(12, 6, 12., 'C12')
O16 = elements(16, 8, 15.994, 'O16')
Ne20 = elements( 20, 10, 20.1797, 'Ne20')

# IME

Mg24 = elements(24, 12, 24.3050, 'Mg24')
Si28 = elements(28, 14, 27.976, 'Si28')
S32 = elements(32, 16, 31.972, 'S32')
Ca40 = elements(40, 20, 39.962, 'Ca40')

# IGE 

Fe54 = elements(54, 26, 53.939, 'Fe54')
Ni58 = elements(58, 28, 57.9353429, 'Ni58')

# Ni56

Ni56 = elements(56, 28, 55.940, 'Ni56')

In [26]:
# Ejecta Layers

co = Layers({C12: 0.45, O16: 0.5, Ne20: 0.05})

ime = Layers({Si28: 0.50, S32: 0.3, Ca40: 0.10, Ar36: 0.08})

ige = Layers({Fe54: 0.5, Ni58: 0.5})

ni = Layers({Ni56: 1.0})

In [30]:
ime

Layer:
	Si28: A=28, Z=14, weight=27.976 abundance = 5.102e-01
	S32: A=32, Z=16, weight=31.972 abundance = 3.061e-01
	Ca40: A=40, Z=20, weight=39.962 abundance = 1.020e-01
	Ar36: A=36, Z=18, weight=35.967 abundance = 8.163e-02